In [1]:
cd ..

## プレスリリースデータの整形

In [9]:
wc -l 20*.csv

   24001 2003.csv
   27568 2004.csv
   29311 2005.csv
       0 2006.csv
   28611 2007.csv
   29947 2008.csv
       0 2009.csv
       0 2010.csv
   29830 2011.csv
   27031 2012.csv
   25100 2013.csv
 1129970 2014.csv
 1351369 total


形式がおかしいデータが混ざっているので注意しつつ必要なデータを抽出

In [11]:
for DATA in 2003.csv 2004.csv 2005.csv 2007.csv 2008.csv 2011.csv 2012.csv 2013.csv; do awk -F'","' '{if (NF != 8) print $0}' ${DATA}; done

articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres
articleid,date,media,length,headline,bodysub,genrel,genres


In [13]:
for DATA in 2003.csv 2004.csv 2005.csv 2007.csv 2008.csv 2011.csv 2012.csv 2013.csv; do awk -F'","' 'NR > 1 {print substr($1, 2) "\t" $2 "\t" $6}' ${DATA} > ${DATA}.for_ar; done

In [14]:
LF=$(printf '\\\012_')
LF=${LF%_}

In [18]:
for DATA in 2006.csv 2010.csv; do sed "s|\"\"|\"${LF}\"|g" ${DATA} | sed "s/genres/genres${LF}/" | awk -F'","' 'NR > 1 {print substr($1, 2) "\t" $2 "\t" $6}' > ${DATA}.for_ar; done

In [16]:
nkf -Lu 2009.csv | awk -F'","' 'NR > 1 {print substr($1, 2) "\t" $2 "\t" $6}' > 2009.csv.for_ar

In [19]:
tr -d "\n" < 2014.csv | sed "s/genres/genres${LF}/" | sed "s|\"\"|\"${LF}\"|g" | awk -F'","' 'NR > 1 {print substr($1, 2) "\t" $2 "\t" $6}' > 2014.csv.for_ar

In [21]:
wc -l *.for_ar

   24001 2003.csv.for_ar
   27568 2004.csv.for_ar
   29311 2005.csv.for_ar
   29785 2006.csv.for_ar
   28611 2007.csv.for_ar
   29947 2008.csv.for_ar
   31110 2009.csv.for_ar
   29656 2010.csv.for_ar
   29830 2011.csv.for_ar
   27031 2012.csv.for_ar
   25100 2013.csv.for_ar
   24515 2014.csv.for_ar
  336465 total


In [22]:
for DATA in *.for_ar; do awk -F'\t' 'NF != 3 {print $0}' ${DATA}; done

タブ区切りにしたことで変な行が無いか確認

In [11]:
cat *.for_ar > pressrelease_all.csv

In [12]:
wc -l pressrelease_all.csv

  336465 pressrelease_all.csv


`pressrelease_all.csv` は全プレスリリースデータを `articleid`, `date`, `bodysub` のタブ区切りで各行に記述したもの

In [1]:
grep "都\|道\|府\|県" pressrelease_all.csv | wc -l

  212727


都道府県の情報を含んでいるものは(高々) $212727/336465 \sim 63\%$

### 同じ企業に関するプレスリリースで、住所が含まれているものと含まれていないものがある例

In [6]:
grep "愛眼" pressrelease_all.csv | wc -l
grep "愛眼" pressrelease_all.csv

       5
NIKPRLRSP302455_07022012	20120207	発表日：2012年2月7日本格スポーツサングラス用度付レンズ加工に対応するＡ．Ｒ．Ｔ．システム（＊）を導入～お客様のニーズに合うデザイン性と機能性の両立を実現～２０１２年２月１０日から、愛眼の全店舗で対応＊ロゴは、添付の関連資料を参照　愛眼株式会社（本社：大阪市天王寺区、代表取締役社長：下條　三千夫）では、本格スポーツサングラス用の度付レンズを加工するＡ．Ｒ．Ｔ．（エー・アール・ティー）システム（＝ＡＧ　Ｒｅｖｉｓｉｏｎ　Ｔｅｃｈ　Ｓｙｓｔｅｍ）を構築し、２０１２年２月１０日より、愛眼の全店舗にて対応できる体制を整えました。【これまでの度付スポーツサングラスの問題点】　現在、本格スポーツサングラスは、より顔・頭にフィットさせて一体感を保ちつつ、同時に目とサングラスの隙間から余計な光や風が進入しないよう、極端にカーブさせたデザインフォルムを持つものが主流となってきています。これまで、そうしたフォルムに充分対応できる度付レンズの加工は難しく、正面視野と周辺視野の間でズレやゆがみが生じたり、出来上がり後にお客様が実際に装着してみると、違和感・距離感のズレを感じたりするケースが少なからずありました。　またスポーツサングラスに度付レンズを入れるサービス自体が、一部の限られた店舗でしか実施されていなかったり、仮に受けられたとしてもコストが高くついたりすることで、お客様にもあまり知られていませんでした。そのため、コンタクトレンズを装着したうえで、スポーツサングラスを使用されている方も多いといわれています。※製品画像などは、添付の関連資料を参照【最適な度数補正、プリズム補正でレンズカーブに起因する違和感などを軽減】　今回愛眼は、フレームメーカーやレンズメーカー、レンズ加工業者らとコラボレーションし、新たにスポーツサングラス用の度付レンズを加工するＡ．Ｒ．Ｔ．システムを構築しました。　まず店舗での受注時に、専用の計測システムを使用して、サングラスフレームのカーブ具合やお客様の眼の位置、視力度数、使用目的（どのスポーツに使用するか）などの個別データを基に、最適な「度数補正」を算出します。加えてレンズメーカーでは製作時に「プリズム補正」をかけることで、レンズのカーブに起因する使用時の違和感・距離感のズレ

In [7]:
grep "愛眼" pressrelease_all.csv | grep "大阪" | wc -l
grep "愛眼" pressrelease_all.csv | grep "大阪"

       3
NIKPRLRSP302455_07022012	20120207	発表日：2012年2月7日本格スポーツサングラス用度付レンズ加工に対応するＡ．Ｒ．Ｔ．システム（＊）を導入～お客様のニーズに合うデザイン性と機能性の両立を実現～２０１２年２月１０日から、愛眼の全店舗で対応＊ロゴは、添付の関連資料を参照　愛眼株式会社（本社：大阪市天王寺区、代表取締役社長：下條　三千夫）では、本格スポーツサングラス用の度付レンズを加工するＡ．Ｒ．Ｔ．（エー・アール・ティー）システム（＝ＡＧ　Ｒｅｖｉｓｉｏｎ　Ｔｅｃｈ　Ｓｙｓｔｅｍ）を構築し、２０１２年２月１０日より、愛眼の全店舗にて対応できる体制を整えました。【これまでの度付スポーツサングラスの問題点】　現在、本格スポーツサングラスは、より顔・頭にフィットさせて一体感を保ちつつ、同時に目とサングラスの隙間から余計な光や風が進入しないよう、極端にカーブさせたデザインフォルムを持つものが主流となってきています。これまで、そうしたフォルムに充分対応できる度付レンズの加工は難しく、正面視野と周辺視野の間でズレやゆがみが生じたり、出来上がり後にお客様が実際に装着してみると、違和感・距離感のズレを感じたりするケースが少なからずありました。　またスポーツサングラスに度付レンズを入れるサービス自体が、一部の限られた店舗でしか実施されていなかったり、仮に受けられたとしてもコストが高くついたりすることで、お客様にもあまり知られていませんでした。そのため、コンタクトレンズを装着したうえで、スポーツサングラスを使用されている方も多いといわれています。※製品画像などは、添付の関連資料を参照【最適な度数補正、プリズム補正でレンズカーブに起因する違和感などを軽減】　今回愛眼は、フレームメーカーやレンズメーカー、レンズ加工業者らとコラボレーションし、新たにスポーツサングラス用の度付レンズを加工するＡ．Ｒ．Ｔ．システムを構築しました。　まず店舗での受注時に、専用の計測システムを使用して、サングラスフレームのカーブ具合やお客様の眼の位置、視力度数、使用目的（どのスポーツに使用するか）などの個別データを基に、最適な「度数補正」を算出します。加えてレンズメーカーでは製作時に「プリズム補正」をかけることで、レンズのカーブに起因する使用時の違和感・距離感のズレ

-> やはり住所も含めた方が良いか？63%という数字は都道府県ではなく市区町村しか書いていないことがあるから？

## 企業名辞書から上場企業のデータを抽出

In [4]:
awk -F'\t' 'NR > 2 && $26 != "" && $26 ~ /[0-9]+/ {print $6 "\t" $13 "\t" $26}' comp_dict.txt > listed_company

In [10]:
wc -l listed_company

    3567 listed_company


`listed_company` は企業名辞書から上場企業(証券コードがあるもの)だけについて、企業名称・住所・証券コードを抽出したもの

## 株価データの整形

In [3]:
find $PWD/kabuka/ -name "D*" | while read DATA; do nkf -Lu ${DATA} | awk -F',' 'BEGIN {first = 1} {if (length($1) == 4) {if ($2 == 11) {add = 1; if (first == 0) {print ""} else {first = 0}; printf $1} else {add = 0}} else if (add == 1) {printf "," $1 ":" $5}} END {print ""}'; done >  kabuka_tse1

In [4]:
wc -l kabuka_tse1

    1788 kabuka_tse1


In [5]:
grep -E "^1002," kabuka_tse1 > market_tse1

`kabuka_tse1` は東証1部上場企業の株価データ、 `market_tse1` はTOPIXデータ